In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
#training data
train_data = pd.read_csv('train.csv')
df_train=train_data.copy()

In [3]:
#test data
test_data = pd.read_csv('test.csv')
df_test=test_data.copy()


# data preprocessing

In [4]:
# data preprocessing
# numerical data: impute using median
# numerical to cat: 'year_built','Year_Factor' 
# combined cat:'year_built'--'old','new';
#             'facility_type'.value_counts()<1000--'other'


In [5]:
# transform some numerical varibles to cat 
# numerical to cat: 'year_built','Year_Factor' 
# for missing 'year_built', we will use median value
df_train['year_built'].fillna(df_train['year_built'].median(),inplace=True)
df_test['year_built'].fillna(df_train['year_built'].median(),inplace=True)

In [6]:
# transform some of the numerical to cat values
df_train['Year_Factor']=df_train['Year_Factor'].astype('object',copy=False)
df_train['year_built']=df_train['year_built'].astype('object',copy=False)
df_test['Year_Factor']=df_test['Year_Factor'].astype('object',copy=False)
df_test['year_built']=df_test['year_built'].astype('object',copy=False)
cat_attribs=df_train.select_dtypes(include=['object']).columns
print(cat_attribs)

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'year_built'],
      dtype='object')


In [7]:
# combine cat varibles 'year_built' and 'facility_type'
df_train['year_built']=df_train['year_built'].apply(lambda x: 'old' if x<=1951 else 'new')

df_test['year_built']=df_test['year_built'].apply(lambda x: 'old' if x<=1951 else 'new')


In [8]:
cat_type=df_train['facility_type'].value_counts()[df_train['facility_type'].value_counts()>= 1000].index

df_train.loc[df_train['facility_type'].isin((df_train['facility_type'].value_counts()
                            [df_train['facility_type'].value_counts() < 1000]).index), 'facility_type'] = 'other'
print(cat_type)   

Index(['Multifamily_Uncategorized', 'Office_Uncategorized',
       'Education_Other_classroom', 'Lodging_Hotel', '2to4_Unit_Building',
       'Commercial_Other', '5plus_Unit_Building', 'Warehouse_Nonrefrigerated',
       'Retail_Uncategorized', 'Education_College_or_university'],
      dtype='object')


In [9]:
# combine the sparse cat for test data as well
df_test.loc[~df_test['facility_type'].isin(cat_type), 'facility_type'] = 'other'
print(sorted(df_train['facility_type'].unique()))
print(sorted(df_test['facility_type'].unique()))
print(sorted(df_train['facility_type'].unique())==sorted(df_test['facility_type'].unique()))

['2to4_Unit_Building', '5plus_Unit_Building', 'Commercial_Other', 'Education_College_or_university', 'Education_Other_classroom', 'Lodging_Hotel', 'Multifamily_Uncategorized', 'Office_Uncategorized', 'Retail_Uncategorized', 'Warehouse_Nonrefrigerated', 'other']
['2to4_Unit_Building', '5plus_Unit_Building', 'Commercial_Other', 'Education_College_or_university', 'Education_Other_classroom', 'Lodging_Hotel', 'Multifamily_Uncategorized', 'Office_Uncategorized', 'Retail_Uncategorized', 'Warehouse_Nonrefrigerated', 'other']
True


In [10]:
df_train.columns

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'floor_area', 'year_built', 'energy_star_rating', 'ELEVATION',
       'january_min_temp', 'january_avg_temp', 'january_max_temp',
       'february_min_temp', 'february_avg_temp', 'february_max_temp',
       'march_min_temp', 'march_avg_temp', 'march_max_temp', 'april_min_temp',
       'april_avg_temp', 'april_max_temp', 'may_min_temp', 'may_avg_temp',
       'may_max_temp', 'june_min_temp', 'june_avg_temp', 'june_max_temp',
       'july_min_temp', 'july_avg_temp', 'july_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'september_min_temp',
       'september_avg_temp', 'september_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'cooling_degree_days',
       'heating_degree_days', 'precipitation_inches', 'snowfall_inc

# pipelines

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [12]:
#training data
y_train=df_train.site_eui
# drop useless variables such as 'id'
X_train=df_train.drop(['site_eui','id'],axis=1)

In [13]:
# test data
X_test=df_test.drop('id',axis=1)
print(X_train.columns==X_test.columns)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [14]:
cat_attr=X_train.select_dtypes(include=['object']).columns
num_attr=X_train.select_dtypes(exclude=['object']).columns
print(cat_attr)
print(num_attr)

Index(['Year_Factor', 'State_Factor', 'building_class', 'facility_type',
       'year_built'],
      dtype='object')
Index(['floor_area', 'energy_star_rating', 'ELEVATION', 'january_min_temp',
       'january_avg_temp', 'january_max_temp', 'february_min_temp',
       'february_avg_temp', 'february_max_temp', 'march_min_temp',
       'march_avg_temp', 'march_max_temp', 'april_min_temp', 'april_avg_temp',
       'april_max_temp', 'may_min_temp', 'may_avg_temp', 'may_max_temp',
       'june_min_temp', 'june_avg_temp', 'june_max_temp', 'july_min_temp',
       'july_avg_temp', 'july_max_temp', 'august_min_temp', 'august_avg_temp',
       'august_max_temp', 'september_min_temp', 'september_avg_temp',
       'september_max_temp', 'october_min_temp', 'october_avg_temp',
       'october_max_temp', 'november_min_temp', 'november_avg_temp',
       'november_max_temp', 'december_min_temp', 'december_avg_temp',
       'december_max_temp', 'cooling_degree_days', 'heating_degree_days',
       'precip

In [15]:
# preprocessing for numerical data
num_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Preprocessing for categorical data
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# buldle preprocessing for numerical and cat data
preprocessor= ColumnTransformer(
    transformers=[
        ('num',num_transformer,num_attr),
        ('cat',cat_transformer,cat_attr)
    ]
)

In [16]:
# define the model
from sklearn.ensemble import RandomForestRegressor
rf_reg=RandomForestRegressor(n_estimators=20, max_leaf_nodes=10,random_state=0)

In [17]:
from sklearn.metrics import mean_squared_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('model',rf_reg)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
prediction=my_pipeline.predict(X_test)


In [19]:
prediction[0:50]

array([ 79.3698976 ,  64.77025127, 260.68416607,  79.3698976 ,
        79.3698976 , 102.60982519, 102.60982519,  66.87224819,
        64.77025127, 121.47882516,  64.77025127,  64.77025127,
        80.56566566,  62.01514428,  61.04769274,  78.22987456,
        80.56566566,  62.01514428, 147.44849157, 147.44849157,
        68.76660045, 144.28511602, 147.44849157, 147.44849157,
       147.44849157, 147.44849157, 147.44849157,  44.9821779 ,
        63.24115876, 147.44849157,  68.76660045, 147.44849157,
       147.44849157, 150.34815907,  63.24115876, 147.44849157,
       102.20119711, 147.44849157,  74.54586813, 147.44849157,
        98.78185417, 124.8040889 , 109.35925424,  63.24115876,
       109.53481806, 109.35925424, 109.35925424, 109.35925424,
       109.35925424,  79.83944858])

In [24]:
df_results = pd.DataFrame(columns = ['id', 'site_eui'])
df_results.id=df_test.id
df_results.site_eui=prediction
df_results.head()
df_results.to_csv('df_prediction.csv', index=None,sep=',')
# the RMSE is 55.504 after I submitted the prediction to Kaggle competition 